In [19]:

# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
data = pd.read_csv('../data/raw/train.csv')

# Display the first few rows
data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [25]:
data['Title'].value_counts()

Title
Mr.          517
Miss.        182
Mrs.         125
Master.       40
Dr.            7
Rev.           6
Mlle.          2
Major.         2
Col.           2
the            1
Capt.          1
Ms.            1
Sir.           1
Lady.          1
Mme.           1
Don.           1
Jonkheer.      1
Name: count, dtype: int64

In [20]:
# extract title, name and family name from Name using a regex
data['Title'] = data['Name'].str.extract(r' ([A-Za-z]+)\. ', expand=False)
data['Title'] = data['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev','Sir', 'Jonkheer'], 'Rare')

data['Title'] = data['Title'].replace(['Mlle', 'Ms'], 'Miss')
data['Title'] = data['Title'].replace('Mme', 'Mrs')
data['Title'] = data['Title'].fillna('Unknown')
data['Title'].value_counts()


Title
Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: count, dtype: int64

In [28]:
# now extract Surname, Title. First Middle (Maiden) from Name
# for example Cumings, Mrs. John Bradley (Florence Briggs Thayer)
# ',' can seperate Surname and Title + First Middle
# then ' ' can seperate Title and First Middle
data['Surname'] = data['Name'].str.split(',').str[0]
data['Title_First_Middle'] = data['Name'].str.split(',').str[1].str.strip()
data['Title'] = data['Title_First_Middle'].str.split(' ').str[0]
data['First_Middle'] = data['Title_First_Middle'].str.split(' ').str[1:].str.join(' ')
data['First_Middle'] = data['First_Middle'].replace('', pd.NA)  # replace empty strings with NaN
data['First_Middle'] = data['First_Middle'].fillna('Unknown')  # fill NaN with 'Unknown'
data['Surname'] = data['Surname'].str.strip()  # remove leading and trailing spaces
data['Title'] = data['Title'].str.strip()  # remove leading and trailing spaces
data['First_Middle'] = data['First_Middle'].str.strip()  # remove leading and trailing spaces
data[['Surname', 'Title', 'First_Middle']].head()



,Surname,Title,First_Middle
0,Braund,Mr.,Owen Harris
1,Cumings,Mrs.,John Bradley (Florence Briggs Thayer)
2,Heikkinen,Miss.,Laina
3,Futrelle,Mrs.,Jacques Heath (Lily May Peel)
4,Allen,Mr.,William Henry


In [33]:
# Assuming df is your DataFrame
data['MaidenName'] = data['Name'].str.extract(r'\((.*?)\)')

# Preview rows that have maiden names
data[['Name', 'MaidenName']].head()


,Name,MaidenName
0,"Braund, Mr. Owen Harris",NaN
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Florence Briggs Thayer
2,"Heikkinen, Miss. Laina",NaN
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Lily May Peel
4,"Allen, Mr. William Henry",NaN


In [45]:
import pandas as pd
import numpy as np

# ---- 0) helpers ----
def _is_unknown(s: pd.Series) -> pd.Series:
    """True for NaN or case/space-insensitive 'Unknown'."""
    return s.isna() | s.astype(str).str.strip().str.lower().eq('unknown')

# ---- 1) CLEAN, GROUP, DISPLAY ----
keys = ['Surname', 'Embarked', 'Pclass']

# (a) A display-friendly Cabin_Grouped: show known cabins; if none, show 'Unknown'
def _agg_cabins_for_display(s: pd.Series) -> str:
    vals = [v for v in s.dropna().unique() if str(v).strip().lower() != 'unknown']
    return 'Unknown' if len(vals) == 0 else ', '.join(vals)

cabin_grouped = (
    data.groupby(keys)['Cabin']
        .agg(_agg_cabins_for_display)
        .reset_index()
        .rename(columns={'Cabin': 'Cabin_Grouped'})
)

# ---- 2) FILL ORIGINAL UNKNOWN/NaN FROM GROUP (when exactly one known cabin exists) ----
# create a "clean" series that ignores Unknown
Cabin_clean = data['Cabin'].mask(_is_unknown(data['Cabin']))

# For each group, if there is exactly ONE known cabin, capture it; otherwise NA
def _single_known(s: pd.Series):
    s = s.dropna().unique()
    return s[0] if len(s) == 1 else np.nan

group_single = (
    data.assign(Cabin_clean=Cabin_clean)
        .groupby(keys)['Cabin_clean']
        .agg(_single_known)
        .reset_index()
        .rename(columns={'Cabin_clean': 'GroupSingleCabin'})
)

# Merge the single-known cabin back and fill Unknown/NaN only where available
data = data.merge(group_single, on=keys, how='left')

mask_unknown = _is_unknown(data['Cabin'])
data['Cabin_Filled'] = data['Cabin']  # start from original
data.loc[mask_unknown & data['GroupSingleCabin'].notna(), 'Cabin_Filled'] = \
    data.loc[mask_unknown & data['GroupSingleCabin'].notna(), 'GroupSingleCabin']

# (optional) if you want Unknown strings where still unknown:
data['Cabin_Filled'] = data['Cabin_Filled'].fillna('Unknown')

# ---- 3) if you also want a grouped view AFTER filling ----
cabin_grouped_after = (
    data.groupby(keys)['Cabin_Filled']
        .agg(_agg_cabins_for_display)
        .reset_index()
        .rename(columns={'Cabin_Filled': 'Cabin_Grouped_After'})
)

# ---- 4) sanity checks (optional) ----
before_known = (~_is_unknown(data['Cabin'])).sum()
after_known  = (~_is_unknown(data['Cabin_Filled'])).sum()
print(f"Known cabins before: {before_known} | after: {after_known} | filled: {after_known - before_known}")

# Example: show groups that changed (like 'C7, Unknown' -> 'C7')
changed = cabin_grouped.merge(cabin_grouped_after, on=keys, how='inner')
print(changed[changed['Cabin_Grouped'] != changed['Cabin_Grouped_After']].head(10))


Known cabins before: 204 | after: 208 | filled: 4
Empty DataFrame
Columns: [Surname, Embarked, Pclass, Cabin_Grouped, Cabin_Grouped_After]
Index: []


In [ ]:
# find rows with

In [37]:
data[data['Surname']=='Smith']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Embarked,Title,Family_name,new_Name,Surname,First_Middle,Title_First_Middle,Middle,Maiden_Name,MaidenName
174,175,0,1,"Smith, Mr. James Clinch",male,56.0,0,0,17764,30.6958,...,C,Mr.,Smith,Mr. James Clinch,Smith,James,Mr. James Clinch,Unknown,Unknown,NaN
260,261,0,3,"Smith, Mr. Thomas",male,NaN,0,0,384461,7.7500,...,Q,Mr.,Smith,Mr. Thomas,Smith,Thomas,Mr. Thomas,Unknown,Unknown,NaN
284,285,0,1,"Smith, Mr. Richard William",male,NaN,0,0,113056,26.0000,...,S,Mr.,Smith,Mr. Richard William,Smith,Richard,Mr. Richard William,Unknown,Unknown,NaN
346,347,1,2,"Smith, Miss. Marion Elsie",female,40.0,0,0,31418,13.0000,...,S,Miss.,Smith,Miss. Marion Elsie,Smith,Marion,Miss. Marion Elsie,Unknown,Unknown,NaN
